# Various stress tests to see if instance and services response adequately

## Inputs and Configuration

In [1]:
import os
import random
import requests
import time

PAVICS_HOST = os.getenv("PAVICS_HOST", "pavics.ouranos.ca").rstrip("/")
if not PAVICS_HOST:
    raise ValueError("Cannot run test without a PAVICS_HOST value.")

PAVICS_URL = f"https://{PAVICS_HOST}"
VERIFY_SSL = True if "DISABLE_VERIFY_SSL" not in os.environ else False
MAGPIE_URL = PAVICS_URL + "/magpie"
TWITCHER_PROXY = "/twitcher/ows/proxy"
TWITCHER_URL = PAVICS_URL + TWITCHER_PROXY

## Utilities

In [8]:
def stress_test_requests(url, runs=100, code=200, method="GET", delays=True, max_err_code=0, max_avg_time=None, **req_kwargs):
    """
    Executes the request for the number of demanded runs and validates the expected status is always returned.
    
    Outputs the results of each request and a summary of their execution time.
    If requested, also validates that all responses were returned on average faster than the maximum allowed time.
    
    :returns: 
        - 0 for no error (success)
        - 1 for error code failure
        - 2 for max-avg-time failure
    """
    print(f"\nStress Test with [{runs}] calls to [{url}]")
    codes = []
    times = []
    delta = [0.] + [float((random.randint(1, 100) / 1000) if delays else 0) for _ in range(1, runs)]
    char = len(str(runs))
    columns = ["Index", "Codes", "Delta", "Times"]
    r = max(len(columns[0]), char)
    w = 10
    header = "".join(f"{c:>{w if i else r}}" for i, c in enumerate(columns))
    for i in range(runs):
        if not i % 10:
            print(f"Progress: {i:>{char}}/{runs}")
        start = time.perf_counter()
        resp = requests.request(method, url, **req_kwargs)
        times.append(time.perf_counter() - start)
        codes.append(resp.status_code)
        if i == runs:
            break
        if delta[i]:
            time.sleep(delta[i])
    data = [f"{_:>{r}}{c:>{w}}{d:>{w-1}.3f}s{t:>{w-1}.3f}s"
            for _, (c, d, t) in enumerate(zip(codes, delta, times))]
    print(f"\n{header}\n" + "\n".join(data))
    avg_time = sum(times) / runs
    min_time = min(times)
    max_time = max(times)
    print("Times:")
    print(f"  min: {min_time:.3f}s")
    print(f"  avg: {avg_time:.3f}s" + (f" !!! (>{max_avg_time}s)" if avg_time > max_avg_time else ""))
    print(f"  max: {max_time:.3f}s")
    if max_avg_time and avg_time > max_avg_time:
        return 2
    return 0 if len([c for c in codes if c == code]) >= (runs - max_err_code) else 1


## Tests

In [9]:
# NBVAL_IGNORE_OUTPUT

for bird in ["finch", "flyingpigeon", "raven"]:
    bird_url = f"{TWITCHER_URL}/{bird}/wps?service=wps&request=getcapabilities"
    assert stress_test_requests(bird_url, runs=100, max_avg_time=1) == 0, "Failure condition encountered"


Stress Test with [100] calls to [https://host-140-69.rdext.crim.ca/twitcher/ows/proxy/finch/wps?service=wps&request=getcapabilities]
Progress:   0/100
Progress:  10/100
Progress:  20/100
Progress:  30/100
Progress:  40/100
Progress:  50/100
Progress:  60/100
Progress:  70/100
Progress:  80/100
Progress:  90/100

Index     Codes     Delta     Times
    0       200    0.000s    0.568s
    1       200    0.073s    0.377s
    2       200    0.004s    0.528s
    3       200    0.024s    0.444s
    4       200    0.040s    0.451s
    5       200    0.050s    0.445s
    6       200    0.015s    0.496s
    7       200    0.019s    0.396s
    8       200    0.084s    0.651s
    9       200    0.056s    1.032s
   10       200    0.098s    0.992s
   11       200    0.002s    0.631s
   12       200    0.034s    0.488s
   13       200    0.011s    0.416s
   14       200    0.070s    0.570s
   15       200    0.093s    0.483s
   16       200    0.046s    0.380s
   17       200    0.048s    0.390s
 